In [2]:
data = pd.read_csv(train_file, header=0)
data.head(5)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
# https://stackoverflow.com/questions/27697766/understanding-min-df-and-max-df-in-scikit-countvectorizer
# Above reference is for understanding the parameters of tf-idf.

tf = TfidfVectorizer(analyzer='word',lowercase=True, min_df = 2,max_df = 0.9,
                     strip_accents='unicode', stop_words = 'english', sublinear_tf=1)

X =  tf.fit_transform(data["comment_text"])
print("The number of features after applying TF-IDF are:")
print(len(tf.get_feature_names()))
print("The shape of the data is:")
print(X.shape)
print("Ten sample features are:")
tf.get_feature_names()[:10]

The number of features after applying TF-IDF are:
74583
The shape of the data is:
(159571, 74583)
Ten sample features are:


['00',
 '000',
 '0000',
 '00000',
 '000000',
 '0000z',
 '0001',
 '0003',
 '000_bucks',
 '000ft']

In [93]:
# Read the test file
test_data = pd.read_csv(test_file)
test_data.tail(5)

,id,comment_text
153159,fffcd0960ee309b5,". \n i totally agree, this stuff is nothing bu..."
153160,fffd7a9a6eb32c16,== Throw from out field to home plate. == \n\n...
153161,fffda9e8d6fafa9e,""" \n\n == Okinotorishima categories == \n\n I ..."
153162,fffe8f1340a79fc2,""" \n\n == """"One of the founding nations of the..."
153163,ffffce3fb183ee80,""" \n :::Stop already. Your bullshit is not wel..."


In [94]:
X_test = tf.transform(test_data["comment_text"])
print(X_test.shape)

(153164, 74583)


In [95]:
# Predictions for toxic  using Logistic
clf = LogisticRegression(C=3, dual=False)
toxic_labels = list(data["toxic"])
clf.fit(X, toxic_labels)
ans_probs = clf.predict_proba(X)
predictions = clf.predict(X)
print("Log Loss: "+str(log_loss(toxic_labels, ans_probs)))
accuracy = round(accuracy_score(toxic_labels, predictions) *100,2)
print("Accuracy: "+str(accuracy))
confusion_matrix(toxic_labels, predictions)

/home/manoj/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Log Loss: 0.07795272249806547
Accuracy: 97.05


array([[143589,    688],
       [  4026,  11268]])

In [79]:
doc = 0
feature_names = tf.get_feature_names()
feature_index = X[doc,:].nonzero()[1]
tfidf_scores = zip(feature_index, [X[doc, x] for x in feature_index])
for w, s in [(feature_names[i], s) for (i, s) in tfidf_scores]:
    print(w, s)

explanation 0.16556591073812327
edits 0.11758377204966428
username 0.1694647006622534
hardcore 0.2432209231302952
metallica 0.2791738288407949
fan 0.1841914893111529
reverted 0.1423831296890881
weren 0.20067679562710994
vandalisms 0.2762286662160805
just 0.08624045494546462
closure 0.2487963967573361
gas 0.22059674408811816
voted 0.21349206599775641
new 0.11554577850801136
york 0.1865334584329208
dolls 0.28749813675315267
fac 0.21480258987499007
don 0.0914176988445954
remove 0.1320049003396883
template 0.15023574657404387
talk 0.08044249121071409
page 0.07950718847788701
retired 0.22073355658683494
89 0.21413990924274262
205 0.22947222656189137
38 0.20363420516848946
27 0.17834616234619227


In [96]:
toxic_test = clf.predict_proba(X_test)

test_data["toxic"] = pd.Series(toxic_test[:,1])

In [97]:
# Predictions for severe_toxic using Logistic Regression
clf = LogisticRegression(C=3, dual=False)
severe_toxic_labels = list(data["severe_toxic"])
clf.fit(X, severe_toxic_labels)
ans_probs = clf.predict_proba(X)
predictions = clf.predict(X)
print("Log Loss: "+str(log_loss(severe_toxic_labels, ans_probs)))
accuracy = round(accuracy_score(severe_toxic_labels, predictions) *100,2)
print("Accuracy: "+str(accuracy))
confusion_matrix(severe_toxic_labels, predictions)

/home/manoj/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Log Loss: 0.018669581299302088
Accuracy: 99.2


array([[157740,    236],
       [  1045,    550]])

In [98]:
severe_toxic_test = clf.predict_proba(X_test)

test_data["severe_toxic"] = pd.Series(severe_toxic_test[:,1])

In [99]:
# Predictions for obscene using Logistic Regression
clf = LogisticRegression(C=3, dual=False)
obscene_labels = list(data["obscene"])
clf.fit(X, obscene_labels)
ans_probs = clf.predict_proba(X)
predictions = clf.predict(X)
print("Log Loss: "+str(log_loss(obscene_labels, ans_probs)))
accuracy = round(accuracy_score(obscene_labels, predictions) *100,2)
print("Accuracy: "+str(accuracy))
confusion_matrix(obscene_labels, predictions)

/home/manoj/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Log Loss: 0.04070735065910144
Accuracy: 98.44


array([[150661,    461],
       [  2031,   6418]])

In [100]:
obscene_test = clf.predict_proba(X_test)

test_data["obscene"] = pd.Series(obscene_test[:,1])

In [101]:
# Predictions for threat using Logistic Regression
clf = LogisticRegression(C=3, dual=False)
threat_labels = list(data["threat"])
clf.fit(X, threat_labels)
ans_probs = clf.predict_proba(X)
predictions = clf.predict(X)
print("Log Loss: "+str(log_loss(threat_labels, ans_probs)))
accuracy = round(accuracy_score(threat_labels, predictions) *100,2)
print("Accuracy: "+str(accuracy))
confusion_matrix(threat_labels, predictions)

/home/manoj/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Log Loss: 0.006377542155268138
Accuracy: 99.76


array([[159058,     35],
       [   340,    138]])

In [102]:
threat_test = clf.predict_proba(X_test)
test_data["threat"] = pd.Series(threat_test[:,1])

In [103]:
# Predictions for insult using Logistic Regression
clf = LogisticRegression(C=3, dual=False)
insult_labels = list(data["insult"])
clf.fit(X, insult_labels)
ans_probs = clf.predict_proba(X)
predictions = clf.predict(X)
print("Log Loss: "+str(log_loss(insult_labels, ans_probs)))
accuracy = round(accuracy_score(insult_labels, predictions) *100,2)
print("Accuracy: "+str(accuracy))
confusion_matrix(insult_labels, predictions)

/home/manoj/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Log Loss: 0.05516407100078059
Accuracy: 97.81


array([[150958,    736],
       [  2758,   5119]])

In [104]:
insult_test = clf.predict_proba(X_test)
test_data["insult"] = pd.Series(insult_test[:,1])

In [105]:
# Predictions for identity_hate using Logistic Regression
clf = LogisticRegression(C=3, dual=False)
identity_hate_labels = list(data["identity_hate"])
clf.fit(X, identity_hate_labels)
ans_probs = clf.predict_proba(X)
predictions = clf.predict(X)
print("Log Loss: "+str(log_loss(identity_hate_labels, ans_probs)))
accuracy = round(accuracy_score(identity_hate_labels, predictions) *100,2)
print("Accuracy: "+str(accuracy))
confusion_matrix(identity_hate_labels, predictions)

/home/manoj/.local/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Log Loss: 0.01667598497594871
Accuracy: 99.35


array([[158066,    100],
       [   938,    467]])

In [106]:
identity_hate_test = clf.predict_proba(X_test)
test_data["identity_hate"] = pd.Series(identity_hate_test[:,1])

In [107]:
del test_data["comment_text"]

In [108]:
test_data.to_csv("submission_3.csv", index=False) # 0.9755-Public 0.9763-Private